# Hadoop
---

* Es un framework para procesar grandes volúmenes de datos en un ambiente distribuido
* Actualmente pertenece a Apache
* Incluye sistema de archivos distribuidos (HDFS)
* Tolerante a fallas
* Diseñado para el procesamiento off-line de los datos (procesamiento batch)

## Componentes

* Common (I/O, serialización, RPC)
* HDFS (file system distribuido)
* Zookeeper (servicio de coordinación de procesos)
* MapReduce (modelo de procesamiento de datos)
* Pig (lenguaje de scripting sobre MapReduce)
* Cascading (framework que simplifica el uso de MapReduce)
* Hive (lenguaje basado en SQL)

## DFS

* Sistema de archivos distribuidos
* Ofrece transparencia al usuario permitiendo operar con todos los archivos del cluster a través del file system distribuido
* Un mismo archivo podría estar almacenado en varias computadoras
* Hay varios:
    * HDFS
    * HFTP
    * HSFTP
    * HAR
    * FTP
    * S3

## HDFS

* Sistema de archivos distribuidos Hadoop
* Todos los archivos se dividen en bloques del mismo tamaño (64MB por defecto, aunque es configurable)
* Los bloques pueden estar físicamente en cualquier computadora
* Permite la réplica de bloques para optimización y recupero de fallas


## Procesos del HDFS

* Namenode
    * Maneja el árbol del filesystem y los metadatos de cada archivo y carpeta
    * Conoce para cada bloque del FS que datanode lo maneja
    * Vínculo con el filesystem del SO
* Datanode del SO
    * Son lo que llevan a cabo la lectura y escritura de los bloques en el filesystem
    * Lleva a cabo la creación, borrado y replicado de los bloques
* Secondary namenode: realiza tareas auxiliares al name node

## Comandos HDFS

* HDFS permite crear, borrar, renombrar archivos y carpetas dentro del FS distribuido
* Copiar un archivo del FS local al HDFS
* Copiar un archivo del HDFS al FS local

```
hadoop$ sbin/start-all.sh                     # inicia servicios
hadoop$ cd bin
hadoop$ ./hdfs dfs -ls                        # lista archivos
hadoop$ ./hdfs dfs -df -h                     # muestra espacio libre
hadoop$ ./hdfs dfs -put algo.txt datos/       # guarda archivo: origen -> destino
hadoop$ ./hdfs dfs -get data3.txt c:/win/user # descarga archivo: origen -> destino
hadoop$ ./hdfs dfs -cat algo.txt | more       # visualiza contenido del archivo q: para salir
```

## Componentes Hadoop

* En Hadoop la administración de los procesos que se ejecutan en el cluster la lleva a cabo un framework llamado Yarn MapReduce
* CLUSTER: conjunto de nodos/máquinas conectadas en red
* Básicamente Yarn realiza los trabajos usando dos procesos diferentes:
    * **Job tracker**: maneja todos los trabajos a ser procesados. Tiene en cuenta el mapa del cluster al momento de crear los procesos Task
    * **Task tracker**: son los encargados de realizar el procesamiento de los datos

## MapReduce

* Es un framework para distribuir tareas en múltiples nodos
* TAREA --> CLUSTER (divide y venceras) distribuye a muchos nodos (físicos o virtuales)
* Ventajas
    * Paralelización y distribución de tareas automática
    * Escalable
    * Tolerante a fallos
    * Monitoreo y capacidad de seguridad
    * Flexibilidad de programación (Java, Python, C#, Ruby, C++)
    * Abstracción al programador

## Fase MapReduce

* **Fase map**: en la que los datos de entrada son procesados, uno a uno, y **transformados** en un conjunto intermedio de datos.
* **Fase reduce**: se reúnen los resultados intermedios y se reducen a un conjunto de datos resumidos, que es el resultado final de la tarea. Se aplican reglas (contar o compactar para que se vea poco)

## Ejemplo MapReduce

* Map
Pedirle a cada nodo que sume y cuente sus datos

* Reduce

```
acum = 0; n = 0
for nodo in cluster:
    acum = acum + nodo.acum
    n = n + nodo.n
promedio = acum / n
```

* La unidad de trabajo de MapReduce es un Job
* Un Job se divide en una tarea map y una tarea reduce
* Los Jobs de MapReduce son controlados por un daemon conocido como JobTracker, el cual reside en el "nodo master"
* Los clientes envían Jobs MapReduce al JobTracker y este distribuye la tarea usando otros nodos del cluster
* Esos nodos se conocen como TaskTracker y son responsables de la ejecución de la tarea asignada y reportar el progreso al JobTracker

## Pyspark

* Herramienta que permite hacer procesamiento

In [3]:
!pip install -q pyspark

In [1]:
import os
os.environ['JAVA_HOME'] = '/Library/Java/JavaVirtualMachines/openjdk-17.jdk/Contents/Home'

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.master("local[*]").getOrCreate() # con * le digo que use todos los nucleos de mi máquina

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/08/10 20:09:04 WARN Utils: Your hostname, MacBook-Pro-de-Pablo.local, resolves to a loopback address: 127.0.0.1; using 192.168.0.84 instead (on interface en0)
25/08/10 20:09:04 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/08/10 20:09:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
rdd = spark.sparkContext.parallelize([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

* RDD (es una lista)
    * R: Robusto
    * D: Distribuido
    * D: Dataset

In [5]:
rdd.collect()

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [6]:
rdd.count()

10

In [7]:
rdd.getNumPartitions()

8

In [8]:
rdd.stats() # se está haciendo en forma paralela

(count: 10, mean: 5.5, stdev: 2.8722813232690143, max: 10.0, min: 1.0)

In [9]:
rdd.max()

10

In [10]:
rdd.histogram(2)

([1.0, 5.5, 10], [5, 5])

In [11]:
# mapeo sumar 1
def sumar_uno(elemento):
    return elemento + 1

nuevo_rdd = rdd.map(sumar_uno) 
nuevo_rdd.collect()

def reducir(x, y):
    return x + y

nuevo_rdd.reduce(reducir)

65

In [12]:
# obtener la suma de todos los pares con map-reduce
def pares(elemento):
    if elemento % 2 == 0:
        return elemento
    return 0

def sumar_pares(x, y):
    return x + y

nuevo_rdd = rdd.map(pares)
nuevo_rdd.collect()

nuevo_rdd.reduce(sumar_pares)

30

In [13]:
frase = spark.sparkContext.textFile('texto.txt') # frase es un rdd

In [15]:
frase.collect()

['Sed ut perspiciatis unde omnis iste natus error sit voluptatem accusantium doloremque laudantium, totam rem aperiam, eaque ipsa quae ab illo inventore veritatis et quasi architecto beatae vitae dicta sunt explicabo. Nemo enim ipsam voluptatem quia voluptas sit aspernatur aut odit aut fugit, sed quia consequuntur magni dolores eos qui ratione voluptatem sequi nesciunt. Neque porro quisquam est, qui dolorem ipsum quia dolor sit amet, consectetur, adipisci velit, sed quia non numquam eius modi tempora incidunt ut labore et dolore magnam aliquam quaerat voluptatem. Ut enim ad minima veniam, quis nostrum exercitationem ullam corporis suscipit laboriosam, nisi ut aliquid ex ea commodi consequatur? Quis autem vel eum iure reprehenderit qui in ea voluptate velit esse quam nihil molestiae consequatur, vel illum qui dolorem eum fugiat quo voluptas nulla pariatur?']

In [16]:
def limpiar(elemento):
    return elemento.split(' ')

def agregar_claves(elemento):
    return (elemento, 1)

frase1 = frase.flatMap(limpiar)  # flatMap: siempre te da una lista
con_claves = frase1.map(agregar_claves)
final = con_claves.reduceByKey(reducir)
final.collect()

[('ut', 3),
 ('perspiciatis', 1),
 ('unde', 1),
 ('omnis', 1),
 ('natus', 1),
 ('error', 1),
 ('doloremque', 1),
 ('laudantium,', 1),
 ('totam', 1),
 ('aperiam,', 1),
 ('quae', 1),
 ('illo', 1),
 ('inventore', 1),
 ('et', 2),
 ('architecto', 1),
 ('dicta', 1),
 ('Nemo', 1),
 ('enim', 2),
 ('aspernatur', 1),
 ('aut', 2),
 ('sed', 2),
 ('eos', 1),
 ('qui', 4),
 ('ratione', 1),
 ('porro', 1),
 ('dolorem', 2),
 ('dolor', 1),
 ('consectetur,', 1),
 ('non', 1),
 ('numquam', 1),
 ('eius', 1),
 ('modi', 1),
 ('tempora', 1),
 ('incidunt', 1),
 ('dolore', 1),
 ('magnam', 1),
 ('voluptatem.', 1),
 ('Ut', 1),
 ('veniam,', 1),
 ('quis', 1),
 ('nostrum', 1),
 ('exercitationem', 1),
 ('ullam', 1),
 ('corporis', 1),
 ('nisi', 1),
 ('ea', 2),
 ('commodi', 1),
 ('autem', 1),
 ('vel', 2),
 ('velit', 1),
 ('esse', 1),
 ('nihil', 1),
 ('molestiae', 1),
 ('consequatur,', 1),
 ('illum', 1),
 ('fugiat', 1),
 ('quo', 1),
 ('nulla', 1),
 ('pariatur?', 1),
 ('Sed', 1),
 ('iste', 1),
 ('sit', 3),
 ('voluptatem', 

In [17]:
final.saveAsTextFile('salida') # guarda en partes
final.coalesce(1).saveAsTextFile('salida1') # en un solo archivo